In [6]:
import urllib.request, json, datetime, time,pytz,csv
import requests
from requests.structures import CaseInsensitiveDict
with open('keys.json') as json_file:
    keys = json.load(json_file)
trainkey = keys["trainkey"]
buskey = keys["buskey"]

#CTA bus and train trackers require an API key from the CTA.
#the following links have how to get an API key for bus and train trackers:
#https://www.transitchicago.com/developers/bustracker/
#https://www.transitchicago.com/developers/ttdocs/

#see the keys_sample.json file for proper formatting

directions = {"East":"1",
             "North":"2",
             "South":"3",
             "West":"4"}
#defines Pace direction numbers

In [2]:
def fetchCTAbuses(busstops):
    if busstops == "":
        return
    busurl = "http://www.ctabustracker.com/bustime/api/v2/getpredictions?key="+buskey+busstops+"&format=json"
    with urllib.request.urlopen(busurl) as url:
        bus_info = json.loads(url.read().decode())
    return bus_info

def parsebus(desired_order, bus_ETA, bus_route, bus_stop):
    busdata = []
    ETAsbyroute = [[] for i in range(len(desired_order))]
    stopsbyroute = [""] * len(desired_order)
    for j in range(0,len(desired_order)):
        for i in range(0,len(bus_ETA)):
            if desired_order[j] == bus_route[i]:
                ETAsbyroute[j].append(bus_ETA[i])
                stopsbyroute[j] = bus_stop[i]

    for i in range(0,len(desired_order)):
        output = []
        arrivals = ""
        if len(ETAsbyroute[i]) == 0:
            continue
        output.append(desired_order[i])
        arrivals += ETAsbyroute[i][0]
        if len(ETAsbyroute[i]) > 1:
            for j in range(1,len(ETAsbyroute[i])):
                arrivals += ", " + ETAsbyroute[i][j]
        output.append(arrivals)
        output.append(stopsbyroute[i])
        busdata.append(output)
    return busdata

def CTAbusinfo(bus_info, desired_order, time_min):
    if bus_info == None:
        return [],[],[],[]
    if "prd" not in bus_info["bustime-response"]:
        return [],[],[],[]
    #bus_info["bustime-response"]["prd"] += (bus_info2["bustime-response"]["prd"])
    #parse northbound and southbound buses
    southbus_route,southbus_ETA,southbus_stop,southbus_dest = [],[],[],[]
    northbus_route,northbus_ETA,northbus_stop,northbus_dest = [],[],[],[]
    westbus_route,westbus_ETA,westbus_stop,westbus_dest = [],[],[],[]
    eastbus_route,eastbus_ETA,eastbus_stop,eastbus_dest = [],[],[],[]
    #don't bother with buses that are almost at their stops, or delayed, or northbound 9, or duplicates
    for i, route in enumerate(desired_order):
        timecutoff = time_min[i]
        for bus in bus_info["bustime-response"]["prd"]:
            if bus["rt"] == route:
                if bus["prdctdn"] == "DUE" or bus["prdctdn"] == "DLY":
                    continue #skips if bus is due, since it's too late to catch or delayed with no real ETA
                elif int(bus["prdctdn"]) < timecutoff:
                    continue #skips if bus is too late to catch
                elif bus["rtdir"]=="Southbound":
                    southbus_route.append(bus["rt"])
                    southbus_ETA.append(bus["prdctdn"])
                    if bus["stpid"] == "4866":
                        southbus_stop.append("LSD & W Sheridan")
                    else:
                        southbus_stop.append(bus["stpnm"])
                    southbus_dest.append(bus["des"])
                elif bus["rtdir"]=="Northbound" or (bus["rtdir"]=="Westbound" and bus["rt"]=="97" and bus["stpid"]=="15078"):
                    northbus_route.append(bus["rt"])
                    northbus_ETA.append(bus["prdctdn"])
                    northbus_stop.append(bus["stpnm"])
                    northbus_dest.append(bus["des"])
                elif bus["rtdir"]=="Westbound":
                    westbus_route.append(bus["rt"])
                    westbus_ETA.append(bus["prdctdn"])
                    westbus_stop.append(bus["stpnm"])
                    westbus_dest.append(bus["des"])
                elif bus["rtdir"]=="Eastbound":
                    eastbus_route.append(bus["rt"])
                    eastbus_ETA.append(bus["prdctdn"])
                    eastbus_stop.append(bus["stpnm"])
                    eastbus_dest.append(bus["des"])
    
    northbus = parsebus(desired_order, northbus_ETA, northbus_route, northbus_stop)
    southbus = parsebus(desired_order, southbus_ETA, southbus_route, southbus_stop)
    westbus = parsebus(desired_order, westbus_ETA, westbus_route, westbus_stop)
    eastbus = parsebus(desired_order, eastbus_ETA, eastbus_route, eastbus_stop)

    return northbus, southbus, westbus, eastbus

In [3]:
def pacedatacollector(stopnums):
    busdata,desired_order = [],[]
    southbus_route,southbus_ETA,southbus_stop,southbus_dest = [],[],[],[]
    northbus_route,northbus_ETA,northbus_stop,northbus_dest = [],[],[],[]
    westbus_route,westbus_ETA,westbus_stop,westbus_dest = [],[],[],[]
    eastbus_route,eastbus_ETA,eastbus_stop,eastbus_dest = [],[],[],[]
    for stop in stopnums:
        busdata = busdata + fetchpacebus(stop)
    for bus in busdata:
        if bus['rt'] not in desired_order:
            desired_order.append(bus['rt'])
        if bus["dirID"]=="1":
            eastbus_route.append(bus["rt"])
            eastbus_ETA.append(str(bus["ETA"]))
            eastbus_stop.append(bus["stpnm"])
        elif bus["dirID"]=="2":
            northbus_route.append(bus["rt"])
            northbus_ETA.append(str(bus["ETA"]))
            northbus_stop.append(bus["stpnm"])
        elif bus["dirID"]=="3":
            southbus_route.append(bus["rt"])
            southbus_ETA.append(str(bus["ETA"]))
            southbus_stop.append(bus["stpnm"])
        elif bus["dirID"]=="4":
            westbus_route.append(bus["rt"])
            westbus_ETA.append(str(bus["ETA"]))
            westbus_stop.append(bus["stpnm"])
    
    
    northbus = parsebus(desired_order, northbus_ETA, northbus_route, northbus_stop)
    southbus = parsebus(desired_order, southbus_ETA, southbus_route, southbus_stop)
    westbus = parsebus(desired_order, westbus_ETA, westbus_route, westbus_stop)
    eastbus = parsebus(desired_order, eastbus_ETA, eastbus_route, eastbus_stop)
    
    return northbus, southbus, westbus, eastbus

def pullstopdata(stopnum):
    with open('PACE_stop_data.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row[7] == stopnum:
                stop = {}
                stop['stpid'] = row[7]
                stop["rt"] = row[0]
                stop["dir"] = row[1]+"bound"
                stop['dirID'] = directions[row[1]]
                with open('PACE_route_numbers.csv') as csv_file2:
                    csv_reader2 = csv.reader(csv_file2, delimiter=',')
                    for row2 in csv_reader2:
                        if row2[0] == row[0]:
                            stop['rtID'] = row2[1]
                stop['stpnm'] = row[5]  
                return stop
    return

def fetchpacebus(stopid):
    url = "http://tmweb.pacebus.com/TMWebWatch/Arrivals.aspx/getStopTimes"
    headers = CaseInsensitiveDict()
    headers["Content-Type"] = "application/json"
    busresults = []
    stopdata = pullstopdata(stopid)
    stopheaders = '{"routeID": '+stopdata["rtID"]+',	"directionID": '+stopdata["dirID"]+',	"stopID":	'+stopdata["stpid"]+',	"tpID":	0, "useArrivalTimes":	true}'
    resp = requests.post(url, headers=headers, data=stopheaders)
    updatetime = resp.json()["d"]['updateTime']+resp.json()["d"]['updatePeriod']
    arrivaldata = resp.json()["d"]["routeStops"][0]["stops"][0]["crossings"]
    counter = 0
    for est in arrivaldata:
        bus = stopdata
        if est['predTime']==None:
            bus['predTime'] = est['schedTime']+est['schedPeriod']
        else:
            bus['predTime'] = est['predTime']+est['predPeriod']
        bus['updateTime'] = updatetime
        bus['ETA'] = getpaceETA(bus['predTime'],updatetime)
        bus['count'] = counter
        counter = counter+1
        busresults.append(bus.copy())
    return busresults

def getpaceETA(esttime,updatetime):
    time2 = datetime.datetime.strptime(esttime,'%I:%M%p')
    time1 = datetime.datetime.strptime(updatetime,'%I:%M%p')
    if time1>time2:
        time2 = time2 + datetime.timedelta(days=1)
    return round((time2 - time1).total_seconds()/60)

In [4]:
CTAbusstops = "&stpid=15077,15078,10438,10462"
#bus stop IDs can be obtained from the CTA bus tracker, or by clicking on the google maps location for a bus stop

CTAbus_desired_order = ["97","54A"]
#this is the order CTA buses should be outputted in.  This can be used to exclude buses not desired, by not listing them here
#all CTA buses desired in the output must be included

time_to_stop = [0,0]
#this is the minimum time for each bus.  the times are in minutes, in the same order as routes in desired_order above.
#this can be used to not display buses that are too late to catch, coming from the location where the display would be located.

CTALstops = "&mapid=41680"
#a full list of L stop IDs is in the train tracker API documentation

pacestops = ['21001','20844','35464','35469']

In [7]:
CTAbusdata = fetchCTAbuses(CTAbusstops)
CTAbusoutput = CTAbusinfo(CTAbusdata,CTAbus_desired_order,time_to_stop)
paceoutput = pacedatacollector(pacestops)
allbusout = [[],[],[],[]]
for i in range(0,4):
    if CTAbusoutput[i] != []:
        for bus in CTAbusoutput[i]:
            allbusout[i].append(bus)
    if paceoutput[i] != []:
        for bus in paceoutput[i]:
            allbusout[i].append(bus)

#the format of the output is a list of lists.
#the output is a list of arrivals for each direction.
#with each direction, the ouput is a list of data about each arriving route.
#first is the route number
#then a string with the number of minutes for a bus to arrive (multiple are separated by commas)
#then the name of the stop
            
#this prints the route numbers, minutes to arrive, and stop names generated by the script
print("north: ",allbusout[0])
print("south: ",allbusout[1])
print("west: ",allbusout[2])
print("east: ",allbusout[3])

north:  [['97', '16', 'Oakton & Lacrosse'], ['210', '21, 71', 'Oakton-Skokie CTA Station']]
south:  [['210', '30, 97', 'Oakton-Skokie CTA Station']]
west:  [['226', '60, 120', 'Oakton/Lincoln']]
east:  [['97', '8, 21', 'Oakton & Lacrosse'], ['226', '27, 85', 'Oakton/Lincoln']]
